<a href="https://colab.research.google.com/github/gabrie1-s/Chagas_analysis/blob/cnn/NACA_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Version: for_CNN_1.0.0

In [1]:
%%capture
!pip install neptune-notebooks

In [2]:
%%capture
!jupyter nbextension enable --py neptune-notebooks

In [3]:
!mkdir dataset

In [4]:
%%capture
!pip install neptune-client

In [5]:
import keras
import pdb
import gc
import requests

In [6]:
%%capture
!pip install --upgrade pip
!pip install mat73
!pip install scikit-learn --upgrade
!pip install scipy
!pip install Bayesian-Optimization
!pip install fpdf

In [7]:
from google.colab import drive
import mat73
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import statistics as st
import cv2

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd /content/drive/MyDrive/CNN_Chagas/Chagas/LSTM

/content/drive/MyDrive/CNN_Chagas/Chagas/LSTM


In [12]:
import neptune.new as neptune

run = neptune.init_run(
    project="gabrie1-s/NACA-data-extraction",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1ODlmNWY4ZS0zNmUzLTQ3NWQtYmVkYy05ZmUyMDM4ZGQxMzkifQ==",
    description='''Aplicamos o NACA para eliminar batimentos ruidosos e
    sinais cujo valor do S/R > 15.
    Consideramos 10 batimentos por sinal.Adotamos o procedimento de shifting com shift = 2.
    Retornamos os sinais dos batimentos extraídos pelo NACA, estes são levemente
    diferentes dos batimentos do sinal original''',
    source_files=["NACA_data_extraction.ipynb"],
)  # your credentials

https://app.neptune.ai/gabrie1-s/NACA-data-extraction/e/NAC-18


In [13]:
%cd -

/content


In [14]:
run["code_version"] = "for_CNN_1.0.0"

In [15]:
dataset = mat73.loadmat('/content/drive/MyDrive/CNN_Chagas/Chagas/data/Dataset_Raw.mat')

In [16]:
signals = list(dataset['Dataset_Raw'])

output = []
for i in range(0, len(signals)):
    output.append(signals[i][-1])
    signals[i] = np.delete(signals[i], [-1,-2])
    # signals[i] = signals[i][0:1000]

In [17]:
def discrete_ratio(ratio_set, n):
    classes = list(range(1, n+1))
    ratio = 1.0/n

    for j in range(0, len(ratio_set)):
        for i in range(0, n):
            if ratio_set[j] > ratio*i and ratio_set[j] <= ratio*(i+1):
                ratio_set[j] = int(classes[i])
                break

    return ratio_set

In [18]:
output = discrete_ratio(output, 2)

In [19]:
with open('Sinais_Chagas.txt') as f:
    lines = f.readlines()

idx = []

for i in range(0, len(lines)):
  arq = lines[i][lines[i].find('FileName1 = '):lines[i].find('\n')]
  signal = lines[i][lines[i].find('Signals{'):lines[i].find('\n')]
  signal = signal[signal.find('{')+1 : signal.find('}')]

  if arq != '':
    idx.append(arq[13:-2])
  if signal != '':
     idx.append(signal)

idx = idx[:-3]


pairs = []
for i in range(0, len(idx), 2):
  val = [idx[i+1], idx[i]]
  pairs.append(val)

In [20]:
matriz_pontos = []
for i in pairs:
    url =  str(i[1]) + '_MatrizPontos.mat'
    pontos = sio.loadmat(url)
    matriz_pontos.append(pontos["Matriz_Pontos"])

In [21]:
peaks = []
for sig in matriz_pontos:
  line = []
  for beat in sig:
    line.append(beat[4])
  peaks.append(line)

## Butterworth filter

In [22]:
from scipy.signal import butter, filtfilt

order = 2
fs = 128  # Sampling frequency
fc = 1  # Cutoff frequency
b, a = butter(order, fc, btype='high', fs=fs)

In [23]:
x = []

for signal in signals:
  filtered_signal = filtfilt(b, a, signal)
  x.append(filtered_signal)

signals = x
del x, filtered_signal

## Data Split

In [24]:
indexes = range(len(signals))

In [25]:
m_rem, m_te, x_rem, x_te, p_rem, p_te, y_rem, y_te = train_test_split(matriz_pontos, signals, peaks, output, test_size=0.15, stratify=output, random_state=38)
m_tr, m_va, x_tr, x_va, p_tr, p_va, y_tr, y_va = train_test_split(m_rem, x_rem, p_rem, y_rem, test_size=0.15, stratify=y_rem, random_state=38)

## Medium beats acquisition

In [26]:
def signals_med(array):
    array_size = len(array[0])
    array_qnt = len(array)
    md_vector = []

    for j in range(0, array_size):
        aux = []

        for i in range(0, array_qnt):
            aux.append(array[i][j])

        md_vector.append(st.median(aux))

    return md_vector

In [27]:
def shifting(signal, samples_position, beat_template, peaks):
  samples = []
  shift = 2
  for p_index, p in enumerate(samples_position):
    aux = []
    for i in range(-1*shift, shift+1):
      aux.append(mean_squared_error(beat_template, signal[p[0]+i:p[1]+i]))

    M = aux.index(min(aux)) - shift
    samples_position[p_index] = [p[0]+M, p[1]+M]
    samples.append(signal[p[0]+M : p[1]+M])
    peaks[p_index] += M


  return samples_position, samples, peaks


In [28]:
def del_var(samples, samples_position, beat_template):

    for idx, sample in enumerate(samples):
        val = 0
        for i in range (len(sample)):
            val += (sample[i] - beat_template[i])**2

        val /= len(sample)

        if val > st.variance(beat_template):
            samples.pop(idx)
            samples_position.pop(idx)

    return samples_position, samples

In [29]:
def snr(beat_template, samples):
  var = st.variance(beat_template)
  noise = []

  for sample in samples:
    n = st.variance(np.subtract(beat_template, sample))
    noise.append(n)

  sr = var/st.median(noise)
  return sr

In [35]:
def window(signal, peaks):
    #calculating L
    peaks_difference = [peaks[i+1] - peaks[i] for i in range(len(peaks)-1)]
    L = 1.1*st.median(peaks_difference)


    #obtening beat samples
    #samples position consists in an array of tuples. Each tuples contains the estimate begin and end of a beat
    samples_position = []
    samples = []

    # pdb.set_trace()
    for idx, p in enumerate(peaks):
        samples.append(signal[p-math.floor(L/3) : p+math.ceil(2*L/3)])
        samples_position.append([p-math.floor(L/3) , p+math.ceil(2*L/3)])

    beat_template = signals_med(samples)
    samples_position, samples, peaks = shifting(signal, samples_position, beat_template, peaks)
    samples_position, samples = del_var(samples, samples_position, beat_template)
    sr = snr(beat_template, samples)

    return sr, samples

In [31]:
del dataset, signal

In [32]:
bps = 10
thres = 15

In [47]:
def thres_test(signal, peaks, outputs, bps):
  templates = []
  y = []

  for idx, p in enumerate(peaks):
    sig_1 = []
    for i in range(0, len(p)-bps, bps):
        sr, samples = window(signal[idx], p[i:i+bps])

        if sr > thres:
          samples = np.concatenate(samples)
          sig_1.append(samples)

    if len(sig_1) > 0:
      sig_1 = np.array(sig_1, dtype='object')
      sig_1 = np.concatenate(sig_1)
      templates.append(sig_1)
      y.append(outputs[idx])

  templates = np.array(templates)
  # pdb.set_trace()

  # arr_idxs = noise_rates.argsort()
  # noise_rates = noise_rates[arr_idxs[::-1]]
  # templates = templates[arr_idxs[::-1]]
  # y = y[arr_idxs[::-1]]

  return templates, y

In [48]:
x_tr, y_tr = thres_test(x_tr, p_tr, y_tr, bps)
x_va, y_va = thres_test(x_va, p_va, y_va, bps)
x_te, y_te = thres_test(x_te, p_te, y_te, bps)

<ipython-input-47-3ccf8155a77f>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  templates = np.array(templates)
<ipython-input-47-3ccf8155a77f>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  templates = np.array(templates)
<ipython-input-47-3ccf8155a77f>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  templates = np.array(tem

In [49]:
np.save("dataset/x_tr.npy", x_tr)
np.save("dataset/x_va.npy", x_va)
np.save("dataset/x_te.npy", x_te)

np.save("dataset/y_tr.npy", y_tr)
np.save("dataset/y_va.npy", y_va)
np.save("dataset/y_te.npy", y_te)

In [50]:
run["data"].upload_files("./dataset/*.npy")

In [52]:
run.stop()